In [1]:
import argparse
import logging
import random
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F
from sklearn.cluster import AffinityPropagation
import numpy as np
from tqdm import tqdm

In [40]:
input_file = 'fi_dev.tsv'
output_file = 'fi_dev_paraphrase_mean_nongreedy.tsv'

# both languages
# model_name = "setu4993/LEALLA-large"

## Finnish model
model_name = 'TurkuNLP/sbert-cased-finnish-paraphrase'

## Russian models
# model_name = 'siberian-lang-lab/evenki-russian-parallel-corpora'
# model_name = 'DeepPavlov/rubert-base-cased-sentence'
threshold = 0.3
result_file = 'fi_dev_rubert_mean_nongreedy_score.txt'

In [41]:
NEW_PERIOD = "new"
OLD_PERIOD = "old"
SENSE_ID_COLUMN = "sense_id"
USAGE_ID_COLUMN = "usage_id"
PERIOD_COLUMN = "period"

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.use_deterministic_algorithms(True)
logging.basicConfig(level=logging.INFO)

In [42]:

def load_model(model_name):
    logging.info(f"Loading model {model_name} for sentence embeddings")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model = model.eval()
    logging.info(f"Loaded model {model_name}")
    return tokenizer, model

tokenizer, model = load_model(model_name)


tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/424k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [43]:
# Getting representations for input sentences

def get_embeddings(sentences)  :
    tokenizer_kwargs = {
        "return_tensors": "pt",
        "padding": True,
        "truncation": True,
        "max_length": 256,
    }
    sent_inputs = tokenizer(sentences, **tokenizer_kwargs)
    with torch.no_grad():
        sent_outputs = model(**sent_inputs).pooler_output # new example
    return sent_outputs



#### cluster algorithm

In [44]:
def cluster_affinity(new_numpy, dist='euclidean'):
    if dist == 'euclidean':
      ap = AffinityPropagation(random_state=42,affinity = 'euclidean',max_iter= 500,verbose=True)
      clustering = ap.fit(new_numpy)

    elif dist == 'cos':
      ap = AffinityPropagation(random_state=42,affinity = 'precomputed',max_iter= 500,verbose=True)
      affi_matrix = np.zeros([new_numpy.shape[0],new_numpy.shape[0]])
      for i in range(new_numpy.shape[0]):
        for j in range(new_numpy.shape[0]):
          affi_matrix[i][j] = F.cosine_similarity(torch.tensor(new_numpy[i]),torch.tensor(new_numpy[j]),dim=0)
      clustering = ap.fit(affi_matrix)
    return clustering.labels_


#### NON-GREEDY algorithm

In [45]:
# this function returns a dictionary {new_example: senseID}
def get_exs2senses_from_cluster():
    unique_labels = np.unique(cluster_label)
    similarities = np.zeros((len(unique_labels), len(old_senses))) ## unique_senses problem
    for label in unique_labels: ## labels are 1,2,3,4...
        this_cluster = new_numpy[cluster_label == label]
        cluster_mean_vec = torch.Tensor(this_cluster.mean(axis=0))
        for sense_idx, gloss_embedding in enumerate(old_outputs):
            sim = F.cosine_similarity(cluster_mean_vec, gloss_embedding, dim=0)
            similarities[label, sense_idx] = sim

    # assign old senses to labels where sim > threshold
    exs2senses = {}
    for label in unique_labels:
            closest_sense_id = similarities[label].argmax()
            if similarities[label,closest_sense_id] > threshold:
                found_sense_id = old_senses[closest_sense_id]
            else:
                latin_name = old_senses[0].split("_")[0]
                found_sense_id = f"{latin_name}_novel_{label}"

            examples_indices = np.where(cluster_label == label)[0]
            examples = [new_examples[i] for i in examples_indices]
            for ex in examples:
                exs2senses[ex] = found_sense_id    ## key-value pairs {example: sense_id}
    return exs2senses

#### the main() method

In [46]:
targets = pd.read_csv(input_file, sep="\t")
for target_word in tqdm(targets.word.unique()):

    this_word = targets[targets.word == target_word]
    new,old = this_word[this_word[PERIOD_COLUMN] == NEW_PERIOD],this_word[this_word[PERIOD_COLUMN] == OLD_PERIOD]

    new_examples = new.example.to_list()

    old_senses = old[SENSE_ID_COLUMN].to_list()
    old_examples = old.example.to_list()
    old_gloss = old.gloss.to_list()

    old_sentence = [] ## old examples (or old gloss)
    for i, sense in enumerate(old_senses):
        if isinstance(old_examples[i],str):
          old_sentence.append(old_examples[i])
        else:
          old_sentence.append(old_gloss[i])

    # Get representations for the new examples and old examples/gloss
    new_outputs = get_embeddings(new_examples)
    old_outputs = get_embeddings(old_sentence)

    ## cluster algorithms
    new_numpy = new_outputs.detach().numpy()
    cluster_label = cluster_affinity(new_numpy)

    # Aligning the old and new senses, return {example: sense_id} key-value pairs
    exs2senses = get_exs2senses_from_cluster()

    new_usage_ids = new[USAGE_ID_COLUMN]
    assert len(new_examples) == new_usage_ids.shape[0] ## make sure the counts of new-sense are equal

    for usage_id, example in zip(new_usage_ids, new_examples):
        system_answer = exs2senses[example] # new sense
        row_number = targets[targets[USAGE_ID_COLUMN] == usage_id].index
        targets.loc[row_number, SENSE_ID_COLUMN] = system_answer

logging.info(f"Writing the result to {output_file}")
targets.to_csv(output_file, sep="\t", index=False)


  0%|          | 0/254 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_affinity_propagation.py:53: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
  1%|          | 2/254 [00:03<08:22,  1.99s/it]

Converged after 17 iterations.


  1%|          | 3/254 [00:05<08:52,  2.12s/it]

Converged after 16 iterations.


  2%|▏         | 4/254 [00:09<10:34,  2.54s/it]

...

Converged after 15 iterations.


 99%|█████████▉| 252/254 [20:04<00:03,  1.63s/it]

Converged after 15 iterations.


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_affinity_propagation.py:53: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
100%|██████████| 254/254 [20:14<00:00,  4.78s/it]

Converged after 18 iterations.


##Evaluation

In [47]:
### evaluation codes
import logging
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score, f1_score
from tqdm import tqdm

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

gold_data = pd.read_csv(input_file, sep="\t")
predictions = pd.read_csv(output_file, sep="\t")

assert len(gold_data) == len(predictions)
assert (
        gold_data[gold_data.period == "new"].example.tolist()
        == predictions[predictions.period == "new"].example.tolist()
)

logger.info(f"Data loaded from {input_file} and {output_file}")
logger.info(f"{len(gold_data)} example usages")
logger.info("Computing Adjusted Rand Index and F1 for predicted senses...")

ari_scores = []
f1_scores = []

for targetword in tqdm(gold_data.word.unique()):
    gold_senses = gold_data[
        (gold_data.word == targetword) & (gold_data.period == "new")
        ].sense_id.values
    pred_senses = predictions[
        (predictions.word == targetword) & (predictions.period == "new")
        ].sense_id.values
    ari = adjusted_rand_score(gold_senses, pred_senses)
    ari_scores.append(ari)
    logger.debug(f"ARI for {targetword}: {ari}")
    old_senses = set(
        gold_data[(gold_data.word == targetword) & (gold_data.period == "old")]
            .sense_id.unique()
            .tolist()
    )
    if len(old_senses) == 0:
        logger.info(f"Not computing F1 for {targetword}: no old senses")
        continue
    test_usages = gold_data[
        (gold_data.word == targetword)
        & (gold_data.period == "new")
        & (gold_data.sense_id.isin(old_senses))
        ]
    test_usages_ids = set(test_usages.usage_id.tolist())
    if len(test_usages) == 0:
        test_usages_predicted = predictions[
            (predictions.word == targetword)
            & (predictions.period == "new")
            & (predictions.sense_id.isin(old_senses))
            ]
        if len(test_usages_predicted) == 0:
            f1_scores.append(1.0)
            logger.info(
                f"Macro F1 set to 1.0 for {targetword}: "
                f"no new usages with old senses, and none predicted"
            )
        else:
            f1_scores.append(0.0)
            logger.info(
                f"Macro F1 set to 0.0 for {targetword}: "
                f"old senses predicted when there are none"
            )
        continue
    test_usages_gold_senses = test_usages.sense_id.tolist()
    test_usages_predictions = predictions[predictions.usage_id.isin(test_usages_ids)]
    test_usages_predicted_senses = test_usages_predictions.sense_id.tolist()
    assert len(test_usages_gold_senses) == len(test_usages_predicted_senses)
    test_usages_predicted_senses = [
        "novel" if el not in old_senses else el
        for el in test_usages_predicted_senses
    ]
    f1 = f1_score(
        test_usages_gold_senses,
        test_usages_predicted_senses,
        average="macro",
        zero_division=0.0,
    )
    logger.debug(f"Macro F1 for {targetword}: {f1}")
    f1_scores.append(f1)

average_ari = np.mean(ari_scores)
logger.info(
    f"Average ARI across {len(ari_scores)} target words: {average_ari:0.3f}"
)
average_f1 = np.mean(f1_scores)
logger.info(
    f"Average macro-F1 across {len(f1_scores)} target words: {average_f1:0.3f}"
)


print(f"ARI: {average_ari:0.3f}")
print(f"F1: {average_f1:0.3f}")

# with open(result_file, "w") as out:

#     print(f"ARI: {average_ari:0.3f}", file=out)
#     print(f"F1: {average_f1:0.3f}", file=out)


100%|██████████| 254/254 [00:03<00:00, 80.35it/s]

ARI: 0.571
F1: 0.670


In [ ]:


# def get_sentence_embeddings(outputs: ModelOutput, no_pooling: bool) -> torch.Tensor:
#   if no_pooling:
#       return outputs.last_hidden_state[:, 0, :]
#   return outputs.pooler_output
# from transformers import (
#     AutoModel,
#     AutoTokenizer,
#     BatchEncoding,
#     PreTrainedModel,
#     PreTrainedTokenizerFast,
# )
# from transformers.utils import ModelOutput

# def find_target_id(example: str, target: str, orth: str) -> int:
#     idx = example.lower().find(target)
#     if idx == -1:
#         idx = example.lower().find(orth)
#     return max(0, idx)

# def find_token_id(inputs: BatchEncoding, batch_id: int, target_id: int) -> int:
#     token_id = inputs.char_to_token(batch_id, target_id)
#     if token_id is None:
#         return 1
#     return token_id

# def get_target_embeddings(
#     inputs: BatchEncoding, outputs: ModelOutput, examples: list, target: str, orths: list
# ) -> torch.Tensor:
#     # Find the starting index of the target word in each example sentence; use 0 as a fallback
#     target_ids = [find_target_id(example, target, orth) for example, orth in zip(examples, orths)]
#     # Get the index of the 1st subtoken of the target word
#     token_ids = [find_token_id(inputs, i, j) for i, j in enumerate(target_ids)]
#     # Get the last hidden layer of this subtoken (or the baseline CLS token as a fallback)
    # return torch.stack([outputs.last_hidden_state[i, j, :] for i, j in enumerate(token_ids)])